In [2]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Check this site for the latest download link
# https://www.apache.org/dyn/closer.lua/spark
!wget -q https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xf spark-3.5.2-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j


0% [Working]
            
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]

0% [Connecting to archive.ubuntu.com] [1 InRelease 12.7 kB/129 kB 10%] [Connect
                                                                               
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy I

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
  .master("local[*]") \
  .appName("ITESO-2024-SparkIntroduction") \
  .config("spark.driver.bindAddress","localhost") \
  .config("spark.ui.port","4040") \
  .getOrCreate()

sc = spark.sparkContext

FileNotFoundError: [Errno 2] No such file or directory: '/opt/spark/./bin/spark-submit'

In [ ]:
# Create an RDD with a list of sentences
sentences = sentences = [
    "Lorem ipsum dolor sit amet, consectetur adipiscing elit.",
    "Sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.",
    "Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat.",
    "Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur.",
    "Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.",
    "Curabitur pretium tincidunt lacus. Nulla gravida orci a odio.",
    "Nullam varius, turpis et commodo pharetra, est eros bibendum elit, nec malesuada elit elit vel lectus.",
    "Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium.",
    "Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit.",
    "Sed quia consequuntur magni dolores eos qui ratione voluptatem sequi nesciunt.",
    "Neque porro quisquam est qui dolorem ipsum quia dolor sit amet, consectetur, adipisci velit.",
    "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.",
    "Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat."
]
sentences_rdd = sc.parallelize(sentences)

# Tokenize the sentences into words
words_rdd = sentences_rdd.flatMap(lambda line: line.split())

print(words_rdd.collect())

In [ ]:
# Compute the frequency of each word
word_counts_rdd = words_rdd.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
print(word_counts_rdd.collect())

In [ ]:
# Find the most common word
most_common_word = word_counts_rdd.takeOrdered(1, key=lambda x: -x[1])
print("Most common word:", most_common_word)

In [ ]:
# Compute the average word length
total_length_rdd = words_rdd.map(lambda word: len(word)).reduce(lambda a, b: a + b)
total_words = words_rdd.count()
average_word_length = total_length_rdd / total_words if total_words > 0 else 0
print("Average word length:", average_word_length)

In [ ]:
#Shortest word
length_words_rdd = words_rdd.map(lambda word: (word, len(word)))
shortest_word = length_words_rdd.sortBy(lambda x: x[1])
print(f'Shortest word is: ', shortest_word.first()[0])

In [ ]:
#Longest word
longest_word = length_words_rdd.sortBy(lambda x: -x[1])
print(f'Longest word is: ', longest_word.first()[0])

In [ ]:
# Stop the SparkContext
sc.stop()